In [1]:
'''
Start
 |
 |---> Install Required Libraries (pip install transformers, sentence-transformers, sklearn, pdfplumber, python-docx)
 |
 |---> Import Required Modules (os, glob, pdfplumber, docx, transformers, sentence-transformers, sklearn, numpy)
 |
 |---> Define Function: Extract Text from Documents
 |      |---> If PDF: Use pdfplumber to extract text
 |      |---> If DOCX: Use python-docx to extract text
 |
 |---> Define Function: Summarize Text using NLP
 |      |---> Load Transformer Model (BART)
 |      |---> Process Long Text in Chunks
 |      |---> Generate Summary using Model
 |
 |---> Define Function: Compute Semantic Embeddings
 |      |---> Load Sentence Transformer Model (all-MiniLM-L6-v2)
 |      |---> Convert Summaries into Vector Embeddings
 |
 |---> Define Function: Cluster Documents into Labels
 |      |---> Apply KMeans Clustering on Embeddings
 |      |---> Assign Labels based on Cluster IDs
 |
 |---> Process Each Document in Folder
 |      |---> Extract Text
 |      |---> Summarize Content
 |      |---> Generate Embeddings
 |      |---> Store Data for Clustering
 |
 |---> Perform Clustering on All Document Embeddings
 |      |---> Fit KMeans Model
 |      |---> Assign Cluster Labels to Documents
 |
 |---> Output Results
 |      |---> Print File Name, Summary, and Assigned Label
 |
End
'''

'\nStart\n |\n |---> Install Required Libraries (pip install transformers, sentence-transformers, sklearn, pdfplumber, python-docx)\n |\n |---> Import Required Modules (os, glob, pdfplumber, docx, transformers, sentence-transformers, sklearn, numpy)\n |\n |---> Define Function: Extract Text from Documents\n |      |---> If PDF: Use pdfplumber to extract text\n |      |---> If DOCX: Use python-docx to extract text\n |\n |---> Define Function: Summarize Text using NLP\n |      |---> Load Transformer Model (BART)\n |      |---> Process Long Text in Chunks\n |      |---> Generate Summary using Model\n |\n |---> Define Function: Compute Semantic Embeddings\n |      |---> Load Sentence Transformer Model (all-MiniLM-L6-v2)\n |      |---> Convert Summaries into Vector Embeddings\n |\n |---> Define Function: Cluster Documents into Labels\n |      |---> Apply KMeans Clustering on Embeddings\n |      |---> Assign Labels based on Cluster IDs\n |\n |---> Process Each Document in Folder\n |      |--

In [8]:
!pip install --upgrade pip setuptools nltk jedi>=0.16 scikit-learn transformers sentence-transformers  pdfplumber python-docx

In [16]:
import os
import glob
import pdfplumber
import docx
from transformers import pipeline
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
import numpy as np
from google.colab import drive
from nltk.tokenize import sent_tokenize
import nltk
from sklearn.metrics.pairwise import cosine_similarity
import pdfplumber
import re
import json

nltk.download('punkt')
drive.mount('/content/drive')
training_folder = "/content/drive/MyDrive/lifesciences/training_documents"



def extract_data_from_docx(docx_file):
    data = []
    doc = docx.Document(docx_file)
    current_section = None

    # Process paragraphs and tables together
    elements = doc.paragraphs + doc.tables
    elements.sort(key=lambda el: el._element.getparent().index(el._element))

    for el in elements:
        if isinstance(el, docx.text.paragraph.Paragraph):
            text = el.text.strip()
            # Detect section headers using regex
            if re.match(r'^\d+(\.\d+)*\s+[A-Za-z ]+', text):
                if current_section:
                    data.append(current_section)
                current_section = {"section_name": text, "table_data": []}
        elif isinstance(el, docx.table.Table):
            table_content = []
            header_row = True  # Flag to skip the header row

            for row in el.rows:
                row_text = [cell.text.strip() for cell in row.cells if cell.text.strip()]
                if header_row:
                    header_row = False  # Skip the first row (header row)
                    continue
                if row_text and not re.match(r'^\d+$', row_text[0]):  # Ignore serial number columns
                    table_content.append(row_text)  # Store row-wise table data

            # Assign table data to the latest detected section
            if current_section and table_content:
                current_section["table_data"].extend(table_content)

    # Ensure the last section is added
    if current_section:
        data.append(current_section)

    return data



def extract_data_from_pdf(pdf_file):
    data = []
    current_section = None

    with pdfplumber.open(pdf_file) as pdf:
        # Extract text and tables from all pages
        all_text = ""
        for page in pdf.pages:
            all_text += page.extract_text() + "\n"

        # Split text into paragraphs (assuming paragraphs are separated by newlines)
        paragraphs = all_text.split("\n")

        # Extract tables from the PDF
        tables = []
        for page in pdf.pages:
            tables.append(page.extract_tables())

        # Flatten tables (in case there are multiple tables across pages)
        flattened_tables = [row for page_tables in tables for table in page_tables for row in table]

        # Process paragraphs
        for para in paragraphs:
            text = para.strip()
            if text:
                # Detect section headers using regex (similar to DOCX section matching)
                if re.match(r'^\d+(\.\d+)*\s+[A-Za-z ]+', text):
                    # Add the current section if it exists before starting a new one
                    if current_section:
                        data.append(current_section)
                    current_section = {"section_name": text, "table_data": []}

        # Process tables
        table_content = []
        for table in flattened_tables:
            for row in table:
                row_text = [cell.strip() for cell in row if cell.strip()]
                # Skip rows that just have serial numbers (assumes first column is serial numbers)
                if row_text and not re.match(r'^\d+$', row_text[0]):
                    table_content.append(row_text)

        # Assign table data to the latest detected section
        if current_section and table_content:
            current_section["table_data"].extend(table_content)

        # Ensure the last section is added
        if current_section:
            data.append(current_section)

    return data


# 1️⃣ Extract text from documents
def extract_text_from_doc(file_path):
    text = ""
    if file_path.endswith(".pdf"):
        data = extract_data_from_pdf(file_path)
    elif file_path.endswith(".docx"):
        data = extract_data_from_docx(file_path)
        text = json.dumps(data, ensure_ascii=False, indent=4)
    return text


# 3️⃣ Compute embeddings for the entire document text
encoder = SentenceTransformer("all-MiniLM-L6-v2")

def compute_embedding(text):
    return encoder.encode(text, convert_to_numpy=True)

# 4️⃣ Function to assign meaningful labels based on summary
def assign_meaningful_label(summary):
    # Define keywords for labels
    labels_dict = {
        "Technology": ["data science", "machine learning", "AI", "artificial intelligence", "deep learning", "algorithm"],
        "Life Sciences": ["health", "biotechnology", "medicine", "pharmaceutical", "biology", "genomics", "life science"],
        "Business": ["marketing", "business", "economy", "finance", "growth", "strategy", "management"],
        "Environment": ["climate", "sustainability", "environment", "ecology", "pollution", "green", "conservation"]
    }

    # Convert summary to lower case for comparison
    summary_lower = summary.lower()

    # Check for matching keywords and assign labels
    for label, keywords in labels_dict.items():
        if any(keyword in summary_lower for keyword in keywords):
            return label
    return "Uncategorized"  # Default if no keywords are found

# Main processing function updated for similarity score based on entire document text
def process_documents(folder_path):
    file_paths = glob.glob(os.path.join(folder_path, "*"))

    documents = []
    summaries = []
    embeddings = []

    # Process each document
    for file_path in file_paths:
        text = extract_text_from_doc(file_path)
        embedding = compute_embedding(text)  # Use entire document text for embedding
        label = assign_meaningful_label(text)  # Assign meaningful label based on the document text

        documents.append({"file": file_path, "text": text, "embedding": embedding, "label": label})
        embeddings.append(embedding)

    # Calculate cosine similarity between all document embeddings
    similarity_matrix = cosine_similarity(np.array(embeddings))

    # Print similarity scores between document pairs
    print("\nDocument Similarity Scores (based on entire text):")
    for i in range(len(documents)):
        for j in range(i + 1, len(documents)):
            similarity_score = similarity_matrix[i][j]
            print(f"Similarity between {documents[i]['file']} and {documents[j]['file']}: {similarity_score:.4f}")

    # Cluster based on embeddings (optional)
    embeddings_matrix = np.array(embeddings)
    labels = cluster_documents(embeddings_matrix, num_clusters=2)

    # Assign labels to documents (if clustering is needed)
    for i, doc in enumerate(documents):
        doc["cluster_label"] = f"Cluster {labels[i]}"

    return documents

# Run on a folder containing documents
results = process_documents(training_folder)

# Print results with meaningful labels
for doc in results:
    print(f"File: {doc['file']}")
    print(f"Meaningful Label: {doc['label']}")
    print(f"Cluster Label: {doc['cluster_label']}\n{'-'*40}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Document Similarity Scores (based on entire text):
Similarity between /content/drive/MyDrive/lifesciences/training_documents/ProtonGlow_URS copy.docx and /content/drive/MyDrive/lifesciences/training_documents/ProtonGlow_URS copy.pdf: -0.0061
File: /content/drive/MyDrive/lifesciences/training_documents/ProtonGlow_URS copy.docx
Meaningful Label: Technology
Cluster Label: Cluster 0
----------------------------------------
File: /content/drive/MyDrive/lifesciences/training_documents/ProtonGlow_URS copy.pdf
Meaningful Label: Uncategorized
Cluster Label: Cluster 1
----------------------------------------
